# Bayes Rule

$$
\begin{align}
p(x|y) &= \dfrac{p(y|x)p(x)}{p(y)} \\
p(y) &= \sum_{x'}p(y|x')p(x'), \text{(discrete)} \\
p(y) &= \int p(y|x')p(x')dx', \text{(continuous)}
\end{align}
$$

And for multiple conditional variables:

$$
p(x|y,z) = \dfrac{p(y|x,z)p(x|z)}{p(y|z)}
$$


# Belief

$$
\begin{align}
bel(x_t) & = p(x_t  | z_{1:t}, u_{1:t}) \\
\overline{bel}(x_t) & = p(x_t|z_{1:t-1}, u_{1:t})
\end{align}
$$



# Bayes Filter

![Bayes Filter Psuedo code](../images/bayesFilter.PNG)

* Line 3: Prediction Step
* Line 4: Measurement Step

# 2.4.2 Example

In [1]:
# Example in section 2.4.2 of the book
def bayes_filter(bel_is_open, push, sense_is_open):
    # constants
    p_sOpen_g_isOpen = 0.6
    p_sClose_g_isOpen = 1.0 - p_sOpen_g_isOpen
    p_sOpen_g_isClose = 0.2
    p_sClose_g_isClose = 1.0 - p_sOpen_g_isClose

    p_isOpen_g_push_isOpen = 1.0
    p_isClose_g_push_isOpen = 0.0
    p_isOpen_g_push_isClose = 0.8
    p_isClose_g_push_isClose = 0.2
    
    p_isOpen_g_NoPush_isOpen = 1.0
    p_isClose_g_NoPush_isOpen = 0.0
    p_isOpen_g_NoPush_isClose = 0.0
    p_isClose_g_NoPush_isClose = 1.0
    
    # prediction step
    if push:
        bel_bar_isOpen = bel_is_open * p_isOpen_g_push_isOpen + \
                          (1.0-bel_is_open) * p_isOpen_g_push_isClose
    else: 
        bel_bar_isOpen = bel_is_open * p_isOpen_g_NoPush_isOpen + \
                          (1-bel_is_open) * p_isOpen_g_NoPush_isClose
    bel_bar_isClose = 1- bel_bar_isOpen
    print("bel_bar_isOpen", bel_bar_isOpen)
    
    # measurement step
    if sense_is_open:
        bel_open = bel_bar_isOpen * p_sOpen_g_isOpen
        bel_close = bel_bar_isClose * p_sOpen_g_isClose
    else: 
        bel_open = bel_bar_isOpen * p_sClose_g_isOpen
        bel_close = bel_bar_isClose * p_sClose_g_isClose
    
    norm = bel_open + bel_close
    bel_open, bel_close = bel_open/norm , bel_close/norm
    print("bel_open", bel_open, "bel_close", bel_close)
    return bel_open

bel_open = bayes_filter(0.5, False, True)
bel_open = bayes_filter(bel_open, True, True)

bel_bar_isOpen 0.5
bel_open 0.7499999999999999 bel_close 0.25
bel_bar_isOpen 0.95
bel_open 0.9827586206896551 bel_close 0.017241379310344845


# 2.8 Exercises

## 2.8.1 
$$
\begin{align}
p(\text{measurement}) &= \text{Uniform}(0,3) \\
p(\text{measurement} | \text{sensor is faulty}) &= \text{Uniform(0,1)}, \text{If faulty, it always outputs below 1} \\
p(\text{sensor is faulty at time 0}) &= 0.01
\end{align}
$$

Suppose sensor does n times measurement and every measurement is under 1 m. What is the posterior probability of sensor being faulty for $n=1, 2, ..., 10$

In [2]:
def bayes_filter(bel_faulty):
    
    # prediction:
    p_faulty_g_faulty = 1.0
    p_Nfaulty_g_faulty = 0.0
    p_faulty_g_Nfaulty = 0.0
    p_Nfaulty_g_Nfaulty = 1.0
    
    bel_bar_faulty = bel_faulty * p_faulty_g_faulty + (1-bel_faulty) * p_faulty_g_Nfaulty
    bel_bar_Nfaulty = 1 - bel_bar_faulty
    # basically nothing changes in prediction step.
    # and bel_bar_faulty = bel_faulty
    # because we do not have any action here?
    
    # measurement
    p_measerment_under_1_g_faulty = 1.0
    p_measerment_under_1_g_Nfaulty = 1.0/3.0
    bel_faulty = bel_bar_faulty * p_measerment_under_1_g_faulty
    bel_Nfaulty = bel_bar_Nfaulty * p_measerment_under_1_g_Nfaulty
    
    # normalization
    s = bel_faulty + bel_Nfaulty
    bel_faulty, bel_Nfaulty = bel_faulty/s, bel_Nfaulty/s
    return bel_faulty, bel_Nfaulty

bel_faulty = 0.01
for idx in range(10):
    bel_faulty, bel_Nfaulty= bayes_filter(bel_faulty)
    print (idx, bel_faulty, bel_Nfaulty)

0 0.029411764705882356 0.9705882352941176
1 0.08333333333333336 0.9166666666666666
2 0.21428571428571433 0.7857142857142856
3 0.4500000000000001 0.5499999999999999
4 0.7105263157894738 0.28947368421052616
5 0.8804347826086957 0.1195652173913043
6 0.9566929133858268 0.04330708661417322
7 0.9851351351351352 0.014864864864864838
8 0.9949954504094632 0.005004549590536838
9 0.9983262325015216 0.0016737674984783609


## 2.8.2

Days are either sunny, cloudy, or rainy. The weather transition function is a Markov chain with a given transition table in the book.

### a) 
suppose day 1 is sunny, what is the probability of the following sequence of days: Day2 = cloudy, Day3 = cloudy, Day4 = rainy?

In [3]:
0.2*0.4*0.2

0.016000000000000004

### b)
Write a simulator that can randomly generate sequences of “weathers” from this state transition function.

In [4]:
import numpy as np
from collections import OrderedDict
from typing import List
from enum import Enum

class Weather(Enum):
    SUNNY = 0
    CLOUDY = 1
    RAINY = 2

class StateTransition:
    OPTIONS = [Weather(i) for i in range(3)]
    def __init__(self, state: Weather,transition_probs: List[float]):
        self.transition_probs = transition_probs
        self.state = state
    
    def sample(self):
        return np.random.choice(StateTransition.OPTIONS, p = self.transition_probs)

class MarkovChain:
    def __init__(self):
        self.states = [
            StateTransition(Weather.SUNNY, [0.8,0.2,0.0]),
            StateTransition(Weather.CLOUDY,[0.4,0.4,0.2]),
            StateTransition(Weather.RAINY,[0.2,0.6,0.2])
        ]
    
    def sample(self, state: Weather):
        idx = state.value
        return self.states[idx].sample()

mc = MarkovChain()
state = Weather.RAINY # init state
print(state)
for _ in range (10):
    state = mc.sample(state)
    print(state)

Weather.RAINY
Weather.CLOUDY
Weather.CLOUDY
Weather.CLOUDY
Weather.CLOUDY
Weather.CLOUDY
Weather.SUNNY
Weather.SUNNY
Weather.SUNNY
Weather.SUNNY
Weather.SUNNY


### c)
Use your simulator to determine the stationary distribution of this Markov chain. The stationary distribution measures the probability that a random day will be sunny, cloudy, or rainy.

In [5]:
number_of_runs = 10000
length_of_runs = 100
states = []
for _ in range(number_of_runs):
    state = np.random.choice(StateTransition.OPTIONS)
    mc = MarkovChain()
    for _ in range(length_of_runs):
        state = mc.sample(state)
    states.append(state)

stationary_dist = {}

for opt in StateTransition.OPTIONS:
    count = 0
    for state in states:
        if state == opt:
            count = count + 1
    stationary_dist[opt] = count / len(states)

print(stationary_dist)

{<Weather.SUNNY: 0>: 0.6439, <Weather.CLOUDY: 1>: 0.2864, <Weather.RAINY: 2>: 0.0697}


### d)
calculate the same as above analytically

In [9]:
transitions = np.array([[0.8, 0.2, 0], 
                        [0.4, 0.4, 0.2], 
                        [0.2, 0.6, 0.2]
                       ], dtype = np.float).T
w,v = np.linalg.eig(transitions)
stationary_dist_analytical = (v @ np.diag([1.0, 0., 0.]) @ np.linalg.inv(v))[:,0]
# Why there is an eigenvalue of 1:
# https://math.stackexchange.com/questions/351142/why-markov-matrices-always-have-1-as-an-eigenvalue
print("answer:", stationary_dist_analytical)

answer: [0.64285714 0.28571429 0.07142857]


In [7]:
# from https://github.com/pptacher/probabilistic_robotics/blob/master/ch2_recursive_state_estimation/ch2_recursive_state_estimation.pdf
[9/14.0, 2/7.0, 1/14.0]

[0.6428571428571429, 0.2857142857142857, 0.07142857142857142]

### e)
Caluculate entropy of the stationary dist

$$
H_p = - \sum_{x} p(x) log_2 p(x)
$$

In [10]:
import math
entropy = -1 * sum([p * math.log2(p) for p in stationary_dist_analytical])
print(f'enrtopy {entropy}')

enrtopy 1.1981174211304029
